# End to End Chicken Disease Classification Project

## Stage 3 : Model Training

In [1]:
%pwd

'D:\\Documents\\Projects\\chiicken_disease_classification\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'D:\\Documents\\Projects\\chiicken_disease_classification'

### Update entity

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_model_path: Path
    training_data : Path
    params_epochs: int
    params_batch_size : int
    params_is_augmentation: bool
    params_image_size: list

### Update configuration manager in src config

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILEPATH,
                 params_filepath = PARAMS_FILEPATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        
        training_data = os.path.join(
            self.config.data_ingestion.unzip_dir,
            "Chicken-fecal-images"
        )
        
        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),updated_model_path = Path(prepare_base_model.updated_model_path),
            training_data = training_data,
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_image_size = params.IMAGE_SIZE,
            params_is_augmentation = params.AUGMENTATION
        )
        
        return training_config
        

NameError: name 'CONFIG_FILEPATH' is not defined

### Update components

In [ ]:
import tensorflow as tf

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_model_path
        )
        
    